In [2]:
import os
import numpy as np
import pandas as pd
import scipy as sc


import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
import plotly.express as px

import plotly.graph_objs as go
import plotly.figure_factory as ff
import cufflinks as cf

cf.set_config_file(offline=True, world_readable=False)

### Introduction¶
In this lecture we examine the process of data cleaning and Exploratory Data Analysis (EDA). Often you will acquire or even be given a collection of data in order to conduct some analysis or answer some questions. The first step in using that data is to ensure that it is in the correct form (cleaned) and that you understand its properties and limitations (EDA). Often as you explore data through EDA you will identify additional transformations that may be required before the data is ready for analysis.

In this notebook we obtain crime data from the city of Berkeley's public records. Ultimately, our goal might be to understand policing patterns but before we get there we must first clean and understand the data.

#### Getting the Data
To begin this analysis we want to get data about crimes in Berkeley. Remarkably, the city of Berkeley maintains an Open Data Portal for citizens to access data about the city. We will be examining the:

1. Call Data
2. Stop Data (NEW)

Fortunately, this data is also relatively well document with detailed descriptions of what it contains. Here are summaries of the fields in the data:

In [3]:
from ds100_utils import fetch_and_cache

#### Downloading the Data
Notice that because I record how I got the data in the notebook, others can reproduce this experiment. However, it is worth noting that the data can change. We will want to pay attention to file timestamps.

In [4]:
calls_file = fetch_and_cache("https://data.cityofberkeley.info/api/views/k2nh-s5h5/rows.csv?accessType=DOWNLOAD",
                "calls_for_service.csv", force=False)

Using cached version that was downloaded (UTC): Sat Mar 11 00:22:01 2023


In [5]:
stops_file = fetch_and_cache("https://data.cityofberkeley.info/api/views/4tbf-3yt8/rows.json?accessType=DOWNLOAD",
                "stops.json", force=False)

Using cached version that was downloaded (UTC): Sat Mar 11 00:22:30 2023


#### Exploring the data
Now that we have obtained the data we want to understand its:

1. Structure -- the "shape" of a data file
2. Granularity -- how fine/coarse is each datum
3. Scope -- how (in)complete is the data
4. Temporality -- how is the data situated in time
5. Faithfulness -- how well does the data capture "reality"

##### 1.

In [6]:
print(calls_file, "is",  os.path.getsize(calls_file) / 1e6, "MB")
print(stops_file, "is", os.path.getsize(stops_file) / 1e6, "MB")

data\calls_for_service.csv is 0.87466 MB
data\stops.json is 17.542422 MB


In [7]:
with open(calls_file, "r") as f:
    print(calls_file, "is", sum(1 for l in f), "lines.")

data\calls_for_service.csv is 13457 lines.


In [8]:
with open(stops_file, "r") as f:
    print(stops_file, "is", sum(1 for l in f), "lines.")

data\stops.json is 59822 lines.


Csv and json which suggests that these are comma separated and javascript object files respectively

In [9]:
print(calls_file, "======================")
with open(calls_file, "r") as f:
    for i in range(10):
        print(i, "\t", repr(f.readline()))

data\calls_for_service.csv ======================
0 	 'CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State\n'
1 	 '22036516,THEFT FROM AUTO,08/10/2022 12:00:00 AM,6:10:00 AM,LARCENY - FROM VEHICLE,3,10/13/2022 04:42:00 PM,"Berkeley, CA\n'
2 	 '(37.86988, -122.27054)",,Berkeley,CA\n'
3 	 '22027674,BURGLARY RESIDENTIAL,06/16/2022 12:00:00 AM,8:25:00 PM,BURGLARY - RESIDENTIAL,4,10/13/2022 04:42:00 PM,"1700 BLOCK OXFORD ST\n'
4 	 'Berkeley, CA\n'
5 	 '(37.87674, -122.26665)",1700 BLOCK OXFORD ST,Berkeley,CA\n'
6 	 '22023993,VANDALISM,05/25/2022 12:00:00 AM,8:00:00 AM,VANDALISM,3,10/13/2022 04:42:00 PM,"2400 BLOCK 8TH ST\n'
7 	 'Berkeley, CA\n'
8 	 '(37.86206, -122.29346)",2400 BLOCK 8TH ST,Berkeley,CA\n'
9 	 '22026547,DISTURBANCE,06/10/2022 12:00:00 AM,9:35:00 PM,DISORDERLY CONDUCT,5,10/13/2022 04:42:00 PM,"1300 BLOCK HEARST AVE\n'


In [10]:
print(stops_file, "======================")
with open(stops_file, "r") as f:
    for i in range(10):
        print(i, "\t", repr(f.readline()))

data\stops.json ======================
0 	 '{\n'
1 	 '  "meta" : {\n'
2 	 '    "view" : {\n'
3 	 '      "id" : "4tbf-3yt8",\n'
4 	 '      "name" : "Berkeley PD - Stop Data (Jan 26, 2015 to Sep 30, 2020)",\n'
5 	 '      "assetType" : "dataset",\n'
6 	 '      "attribution" : "City of Berkeley Police Department",\n'
7 	 '      "averageRating" : 0,\n'
8 	 '      "category" : "Public Safety",\n'
9 	 '      "createdAt" : 1588602591,\n'


In [11]:
d = {
  "field1": "value1",
  "field2": ["list", "of", "values"],
  "myfield3": {"is_recursive": True, "a null value": None}
}
print(d['myfield3'])

{'is_recursive': True, 'a null value': None}


In [12]:
calls = pd.read_csv(calls_file)
calls.head()

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State
0,22036516,THEFT FROM AUTO,08/10/2022 12:00:00 AM,6:10:00 AM,LARCENY - FROM VEHICLE,3,10/13/2022 04:42:00 PM,"Berkeley, CA\n(37.86988, -122.27054)",NaN,Berkeley,CA
1,22027674,BURGLARY RESIDENTIAL,06/16/2022 12:00:00 AM,8:25:00 PM,BURGLARY - RESIDENTIAL,4,10/13/2022 04:42:00 PM,"1700 BLOCK OXFORD ST\nBerkeley, CA\n(37.87674,...",1700 BLOCK OXFORD ST,Berkeley,CA
2,22023993,VANDALISM,05/25/2022 12:00:00 AM,8:00:00 AM,VANDALISM,3,10/13/2022 04:42:00 PM,"2400 BLOCK 8TH ST\nBerkeley, CA\n(37.86206, -1...",2400 BLOCK 8TH ST,Berkeley,CA
3,22026547,DISTURBANCE,06/10/2022 12:00:00 AM,9:35:00 PM,DISORDERLY CONDUCT,5,10/13/2022 04:42:00 PM,"1300 BLOCK HEARST AVE\nBerkeley, CA\n(37.87133...",1300 BLOCK HEARST AVE,Berkeley,CA
4,22018126,ASSAULT/BATTERY FEL.,04/20/2022 12:00:00 AM,3:31:00 PM,ASSAULT,3,10/13/2022 04:42:00 PM,"3000 BLOCK SHATTUCK AVE\nBerkeley, CA\n(37.855...",3000 BLOCK SHATTUCK AVE,Berkeley,CA


In [13]:
calls.shape[0]

4490

Preliminary observations on the data?

1. EVENTDT -- Contain the incorrect time stamp
2. EVENTTM -- Contains the time in 24 hour format (What timezone?)
3. CVDOW -- Appears to be some encoding of the day of the week (see data documentation).
4. InDbDate -- Appears to be correctly formatted and appears pretty consistent in time.
5. Block_Location -- Errr, what a mess! newline characters, and Geocoordinates all merged!! Fortunately, this field was "quoted" otherwise we would have had trouble parsing the file. (why?)
6. BLKADDR -- This appears to be the address in Block Location.
7. City and State seem redundant given this is supposed to be the city of Berkeley dataset.

In [14]:
calls.groupby(["City", "State"]).count()

,,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR
City,State,,,,,,,,,
Berkeley,CA,4490,4490,4490,4490,4490,4490,4490,4490,4476


In [15]:
dow = pd.Series(["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"], name="Day")
dow

0       Sunday
1       Monday
2      Tuesday
3    Wednesday
4     Thursday
5       Friday
6     Saturday
Name: Day, dtype: object

In [16]:
df_dow = pd.DataFrame(dow)
# Notice that I am dropping the column if it already exists to
# make it so I can run this cell more than once
# calls & cvsow
calls = pd.merge(calls.drop(columns="Day", errors="ignore"), 
         df_dow, left_on='CVDOW', right_index=True).sort_index()
calls.head()

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day
0,22036516,THEFT FROM AUTO,08/10/2022 12:00:00 AM,6:10:00 AM,LARCENY - FROM VEHICLE,3,10/13/2022 04:42:00 PM,"Berkeley, CA\n(37.86988, -122.27054)",NaN,Berkeley,CA,Wednesday
1,22027674,BURGLARY RESIDENTIAL,06/16/2022 12:00:00 AM,8:25:00 PM,BURGLARY - RESIDENTIAL,4,10/13/2022 04:42:00 PM,"1700 BLOCK OXFORD ST\nBerkeley, CA\n(37.87674,...",1700 BLOCK OXFORD ST,Berkeley,CA,Thursday
2,22023993,VANDALISM,05/25/2022 12:00:00 AM,8:00:00 AM,VANDALISM,3,10/13/2022 04:42:00 PM,"2400 BLOCK 8TH ST\nBerkeley, CA\n(37.86206, -1...",2400 BLOCK 8TH ST,Berkeley,CA,Wednesday
3,22026547,DISTURBANCE,06/10/2022 12:00:00 AM,9:35:00 PM,DISORDERLY CONDUCT,5,10/13/2022 04:42:00 PM,"1300 BLOCK HEARST AVE\nBerkeley, CA\n(37.87133...",1300 BLOCK HEARST AVE,Berkeley,CA,Friday
4,22018126,ASSAULT/BATTERY FEL.,04/20/2022 12:00:00 AM,3:31:00 PM,ASSAULT,3,10/13/2022 04:42:00 PM,"3000 BLOCK SHATTUCK AVE\nBerkeley, CA\n(37.855...",3000 BLOCK SHATTUCK AVE,Berkeley,CA,Wednesday


In [17]:
calls['Block_Location'].head(10)

0                 Berkeley, CA\n(37.86988, -122.27054)
1    1700 BLOCK OXFORD ST\nBerkeley, CA\n(37.87674,...
2    2400 BLOCK 8TH ST\nBerkeley, CA\n(37.86206, -1...
3    1300 BLOCK HEARST AVE\nBerkeley, CA\n(37.87133...
4    3000 BLOCK SHATTUCK AVE\nBerkeley, CA\n(37.855...
5    2400 BLOCK BLAKE ST\nBerkeley, CA\n(37.86397, ...
6    1800 BLOCK 6TH ST\nBerkeley, CA\n(37.87025, -1...
7    2400 BLOCK LE CONTE AVE\nBerkeley, CA\n(37.876...
8    2300 BLOCK DURANT AVE\nBerkeley, CA\n(37.86725...
9    2100 BLOCK 10TH ST\nBerkeley, CA\n(37.86746, -...
Name: Block_Location, dtype: object

In [18]:
calls_lat_lon = (
    # Remove newlines
    calls['Block_Location'].str.replace("\n", "\t") 
    # Extract Lat and Lon using regular expression
    .str.extract(".*\((?P<Lat>\d*\.\d*)\, (?P<Lon>-?\d*\.\d*)\)", expand=True)
)
calls_lat_lon.head(10)

,Lat,Lon
0,37.86988,-122.27054
1,37.87674,-122.26665
2,37.86206,-122.29346
3,37.87133,-122.28656
4,37.85507,-122.26677
5,37.86397,-122.26036
6,37.87025,-122.29843
7,37.87637,-122.26229
8,37.86725,-122.26326
9,37.86746,-122.29285


In [19]:
(~calls_lat_lon.isnull()).mean()

Lat    1.0
Lon    1.0
dtype: float64

In [20]:
# Remove Lat and Lon if they already existed before (reproducible)
calls.drop(["Lat", "Lon"], axis=1, inplace=True, errors="ignore")
# Join in the the latitude and longitude data
calls = calls.merge(calls_lat_lon, left_index=True, right_index=True)
# calls[["Lat", "Lon"]] = calls_lat_lon
# calls.join(calls_lat_lon)
calls.head()

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day,Lat,Lon
0,22036516,THEFT FROM AUTO,08/10/2022 12:00:00 AM,6:10:00 AM,LARCENY - FROM VEHICLE,3,10/13/2022 04:42:00 PM,"Berkeley, CA\n(37.86988, -122.27054)",NaN,Berkeley,CA,Wednesday,37.86988,-122.27054
1,22027674,BURGLARY RESIDENTIAL,06/16/2022 12:00:00 AM,8:25:00 PM,BURGLARY - RESIDENTIAL,4,10/13/2022 04:42:00 PM,"1700 BLOCK OXFORD ST\nBerkeley, CA\n(37.87674,...",1700 BLOCK OXFORD ST,Berkeley,CA,Thursday,37.87674,-122.26665
2,22023993,VANDALISM,05/25/2022 12:00:00 AM,8:00:00 AM,VANDALISM,3,10/13/2022 04:42:00 PM,"2400 BLOCK 8TH ST\nBerkeley, CA\n(37.86206, -1...",2400 BLOCK 8TH ST,Berkeley,CA,Wednesday,37.86206,-122.29346
3,22026547,DISTURBANCE,06/10/2022 12:00:00 AM,9:35:00 PM,DISORDERLY CONDUCT,5,10/13/2022 04:42:00 PM,"1300 BLOCK HEARST AVE\nBerkeley, CA\n(37.87133...",1300 BLOCK HEARST AVE,Berkeley,CA,Friday,37.87133,-122.28656
4,22018126,ASSAULT/BATTERY FEL.,04/20/2022 12:00:00 AM,3:31:00 PM,ASSAULT,3,10/13/2022 04:42:00 PM,"3000 BLOCK SHATTUCK AVE\nBerkeley, CA\n(37.855...",3000 BLOCK SHATTUCK AVE,Berkeley,CA,Wednesday,37.85507,-122.26677


In [21]:
calls[calls['Lat'].isnull()].head(10)

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day,Lat,Lon


json

In [22]:
import json

with open("data/stops.json", "rb") as f:
    stops_json = json.load(f)

In [23]:
type(stops_json)

dict

In [24]:
stops_json.keys()

dict_keys(['meta', 'data'])

In [25]:
stops_json['meta'].keys()

dict_keys(['view'])

In [26]:
stops_json['meta']['view'].keys()

dict_keys(['id', 'name', 'assetType', 'attribution', 'averageRating', 'category', 'createdAt', 'description', 'displayType', 'downloadCount', 'hideFromCatalog', 'hideFromDataJson', 'licenseId', 'newBackend', 'numberOfComments', 'oid', 'provenance', 'publicationAppendEnabled', 'publicationDate', 'publicationGroup', 'publicationStage', 'rowsUpdatedAt', 'rowsUpdatedBy', 'tableId', 'totalTimesRated', 'viewCount', 'viewLastModified', 'viewType', 'approvals', 'clientContext', 'columns', 'grants', 'license', 'metadata', 'owner', 'query', 'rights', 'tableAuthor', 'tags', 'flags'])

In [27]:
print(stops_json['meta']['view']['description'])

This data was extracted from the Department’s Public Safety Server and covers data beginning January 26, 2015.  On January 26, 2015 the department began collecting data pursuant to General Order B-4 (issued December 31, 2014). Under that Order, officers were required to provide certain data after making any detention (vehicle, bicycle, pedestrian, suspicious auto).  This dataset provides information about detentions, including the race, sex, age range, of the person detained; the reason for the stop; the type of enforcement taken (if any), and whether or not a search was conducted.  Also provided are the date, time, location of the detention, as well as the incident number and call for service type.


In [28]:
type(stops_json['meta']['view']['columns'])

list

In [29]:
for c in stops_json['meta']['view']['columns']:
    top = ""
    items = ""
    if "cachedContents" in c:
        top = c["cachedContents"]["top"]
        for i in top:
            items = items + "\n\t" + i["item"] + ":" + i["count"]
    print(c["name"], ":", items)

sid : 
id : 
position : 
created_at : 
created_meta : 
updated_at : 
updated_meta : 
meta : 
CreateDatetime : 
	2015-06-11T06:48:26.000:10
	2020-01-07T08:15:40.000:6
	2020-02-13T08:10:43.000:6
	2017-01-26T11:10:06.000:6
	2016-07-03T23:45:04.000:5
	2016-12-22T23:58:59.000:5
	2017-01-06T00:25:16.000:5
	2016-04-19T00:50:19.000:5
	2015-11-16T07:19:28.000:5
	2016-03-25T01:13:14.000:5
	2015-02-01T19:33:13.000:5
	2016-06-11T19:55:36.000:5
	2015-12-16T23:11:58.000:4
	2016-05-25T01:20:06.000:4
	2016-05-01T01:23:53.000:4
	2018-09-29T00:59:15.000:4
	2016-12-05T00:14:11.000:4
	2018-02-23T00:12:21.000:4
	2015-12-19T02:11:34.000:4
	2015-05-26T10:04:15.000:4
IncidentNumber : 
	2020-00001049:6
	2020-00008547:6
	2016-00075241:5
	2016-00034574:5
	2017-00000828:5
	2016-00039405:5
	2016-00022806:5
	2015-00033576:5
	2015-00033565:5
	2016-00017339:5
	2015-00067321:5
	2015-00006347:5
	2016-00030130:4
	2018-00054850:4
	2016-00042775:4
	2016-00042412:4
	2015-00030246:4
	2015-00073776:4
	2016-00027631:4
	2017-0

In [30]:
for i in range(3):
    print(i, "\t", stops_json['data'][i])

0 	 ['row-4x5i_bhbt.p5i8', '00000000-0000-0000-A5A4-54B7379F0A22', 0, 1667552430, None, 1667552430, None, '{ }', '2016-04-21T21:19:43', '2016-00023488', 'MARTIN LUTHER KING JR WAY / DWIGHT WAY', 'BERKELEY', '37.8633536670806', '-122.272112375695', 'T', 'Black', 'Male', '>40', 'Traffic', 'Warning', 'No Search']
1 	 ['row-7zd2.fzni_26x7', '00000000-0000-0000-5F81-4C8F8669527C', 0, 1622797226, None, 1622797226, None, '{ }', '2018-03-14T16:25:55', '2018-00015116', 'ANTHONY ST / 7TH ST', 'BERKELEY', '37.8522263089015', '-122.291495435525', 'T', 'White', 'Female', '30-39', 'Traffic', 'Citation', 'No Search']
2 	 ['row-ybnd_6bus.53vc', '00000000-0000-0000-BA6C-637C41AA3D15', 0, 1646992836, None, 1646992836, None, '{ }', '2020-02-05T12:33:06', '2020-00006993', 'SACRAMENTO ST / DERBY ST', 'BERKELEY', '37.8587127926943', '-122.28015517143', 'T', 'Black', 'Female', '>40', 'Traffic', 'Warning', 'No Search']


Building a Dataframe from JSON

In the following block of code we:

Translate the JSON records into a dataframe
Remove columns that have no metadata description. This would be a bad idea in general but here we remove these columns since the above analysis suggests that they are unlikely to contain useful information.
Examine the top of the table

In [31]:
# Load the data from JSON and assign column titles
stops = pd.DataFrame(
    stops_json['data'],
    columns=[c['name'] for c in stops_json['meta']['view']['columns']])

stops.head()

,sid,id,position,created_at,created_meta,updated_at,updated_meta,meta,CreateDatetime,IncidentNumber,...,City,Lat,Lon,CallType,Race,Gender,Age,Reason,Enforcement,Car Search
0,row-4x5i_bhbt.p5i8,00000000-0000-0000-A5A4-54B7379F0A22,0,1667552430,None,1667552430,None,{ },2016-04-21T21:19:43,2016-00023488,...,BERKELEY,37.8633536670806,-122.272112375695,T,Black,Male,>40,Traffic,Warning,No Search
1,row-7zd2.fzni_26x7,00000000-0000-0000-5F81-4C8F8669527C,0,1622797226,None,1622797226,None,{ },2018-03-14T16:25:55,2018-00015116,...,BERKELEY,37.8522263089015,-122.291495435525,T,White,Female,30-39,Traffic,Citation,No Search
2,row-ybnd_6bus.53vc,00000000-0000-0000-BA6C-637C41AA3D15,0,1646992836,None,1646992836,None,{ },2020-02-05T12:33:06,2020-00006993,...,BERKELEY,37.8587127926943,-122.28015517143,T,Black,Female,>40,Traffic,Warning,No Search
3,row-dj2m-24xk-aik2,00000000-0000-0000-F03D-FAF659ECC072,0,1653037224,None,1653037224,None,{ },2017-05-31T18:10:49,2017-00031076,...,BERKELEY,37.8704105618947,-122.281937987589,T,Hispanic,Male,18-29,Traffic,Warning,No Search
4,row-m7he.wthe.w7r4,00000000-0000-0000-6CC6-513F3C600645,0,1623402030,None,1623402030,None,{ },2016-11-06T07:15:42,2016-00065741,...,BERKELEY,37.8541134318992,-122.284193275337,1196,White,Male,>40,Investigation,Warning,No Search


In [32]:
stops.columns

Index(['sid', 'id', 'position', 'created_at', 'created_meta', 'updated_at',
       'updated_meta', 'meta', 'CreateDatetime', 'IncidentNumber', 'Address',
       'City', 'Lat', 'Lon', 'CallType', 'Race', 'Gender', 'Age', 'Reason',
       'Enforcement', 'Car Search'],
      dtype='object')

In [33]:
pd.set_option('display.max_columns', 100) 
pd.set_option('display.max_rows', 100) 

In [34]:
stops.head(10)

,sid,id,position,created_at,created_meta,updated_at,updated_meta,meta,CreateDatetime,IncidentNumber,Address,City,Lat,Lon,CallType,Race,Gender,Age,Reason,Enforcement,Car Search
0,row-4x5i_bhbt.p5i8,00000000-0000-0000-A5A4-54B7379F0A22,0,1667552430,None,1667552430,None,{ },2016-04-21T21:19:43,2016-00023488,MARTIN LUTHER KING JR WAY / DWIGHT WAY,BERKELEY,37.8633536670806,-122.272112375695,T,Black,Male,>40,Traffic,Warning,No Search
1,row-7zd2.fzni_26x7,00000000-0000-0000-5F81-4C8F8669527C,0,1622797226,None,1622797226,None,{ },2018-03-14T16:25:55,2018-00015116,ANTHONY ST / 7TH ST,BERKELEY,37.8522263089015,-122.291495435525,T,White,Female,30-39,Traffic,Citation,No Search
2,row-ybnd_6bus.53vc,00000000-0000-0000-BA6C-637C41AA3D15,0,1646992836,None,1646992836,None,{ },2020-02-05T12:33:06,2020-00006993,SACRAMENTO ST / DERBY ST,BERKELEY,37.8587127926943,-122.28015517143,T,Black,Female,>40,Traffic,Warning,No Search
3,row-dj2m-24xk-aik2,00000000-0000-0000-F03D-FAF659ECC072,0,1653037224,None,1653037224,None,{ },2017-05-31T18:10:49,2017-00031076,UNIVERSITY AVE / SACRAMENTO ST,BERKELEY,37.8704105618947,-122.281937987589,T,Hispanic,Male,18-29,Traffic,Warning,No Search
4,row-m7he.wthe.w7r4,00000000-0000-0000-6CC6-513F3C600645,0,1623402030,None,1623402030,None,{ },2016-11-06T07:15:42,2016-00065741,RUSSELL ST / MABEL ST,BERKELEY,37.8541134318992,-122.284193275337,1196,White,Male,>40,Investigation,Warning,No Search
5,row-r383_ubix.9xuy,00000000-0000-0000-CF86-61C4B76076D9,0,1622797226,None,1622797226,None,{ },2018-11-09T10:17:02,2018-00062865,SHATTUCK AVE / CEDAR ST,BERKELEY,37.8784161184258,-122.269103891551,1194,White,Male,18-29,Investigation,Warning,No Search
6,row-p9bk-fwyy~at6c,00000000-0000-0000-3FB5-9F1D6D225BB6,0,1646992836,None,1646992836,None,{ },2015-02-13T08:44:06,2015-00008971,SACRAMENTO ST / DERBY ST,BERKELEY,37.8587127926943,-122.28015517143,T,Hispanic,Male,18-29,Traffic,Citation,No Search
7,row-k46e.j67p~jzt2,00000000-0000-0000-A946-FA8764E3CB02,0,1622797226,None,1622797226,None,{ },2016-05-29T17:42:54,2016-00031669,1614 ASHBY AVE,BERKELEY,37.8534427593358,-122.276192560053,1194B,White,Male,>40,Investigation,Warning,No Search
8,row-9kze_8vgv~9pti,00000000-0000-0000-6798-75260E89DF20,0,1667552430,None,1667552430,None,{ },2017-03-25T15:18:55,2017-00016677,5TH ST / ADDISON ST,BERKELEY,37.8664685710026,-122.298396619921,T,Black,Female,30-39,Traffic,Warning,No Search
9,row-9j8k-64jq-ep2i,00000000-0000-0000-3D5B-DB7FC518208C,0,1653037224,None,1653037224,None,{ },2017-02-19T22:41:32,2017-00009777,UNIVERSITY AVE / SACRAMENTO ST,BERKELEY,37.8704105618947,-122.281937987589,T,Asian,Male,>40,Traffic,Warning,No Search


### EDA

In [35]:
calls.head()

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day,Lat,Lon
0,22036516,THEFT FROM AUTO,08/10/2022 12:00:00 AM,6:10:00 AM,LARCENY - FROM VEHICLE,3,10/13/2022 04:42:00 PM,"Berkeley, CA\n(37.86988, -122.27054)",NaN,Berkeley,CA,Wednesday,37.86988,-122.27054
1,22027674,BURGLARY RESIDENTIAL,06/16/2022 12:00:00 AM,8:25:00 PM,BURGLARY - RESIDENTIAL,4,10/13/2022 04:42:00 PM,"1700 BLOCK OXFORD ST\nBerkeley, CA\n(37.87674,...",1700 BLOCK OXFORD ST,Berkeley,CA,Thursday,37.87674,-122.26665
2,22023993,VANDALISM,05/25/2022 12:00:00 AM,8:00:00 AM,VANDALISM,3,10/13/2022 04:42:00 PM,"2400 BLOCK 8TH ST\nBerkeley, CA\n(37.86206, -1...",2400 BLOCK 8TH ST,Berkeley,CA,Wednesday,37.86206,-122.29346
3,22026547,DISTURBANCE,06/10/2022 12:00:00 AM,9:35:00 PM,DISORDERLY CONDUCT,5,10/13/2022 04:42:00 PM,"1300 BLOCK HEARST AVE\nBerkeley, CA\n(37.87133...",1300 BLOCK HEARST AVE,Berkeley,CA,Friday,37.87133,-122.28656
4,22018126,ASSAULT/BATTERY FEL.,04/20/2022 12:00:00 AM,3:31:00 PM,ASSAULT,3,10/13/2022 04:42:00 PM,"3000 BLOCK SHATTUCK AVE\nBerkeley, CA\n(37.855...",3000 BLOCK SHATTUCK AVE,Berkeley,CA,Wednesday,37.85507,-122.26677


In [36]:
print("There are", calls['CASENO'].unique().shape[0], "unique case numbers.")
print("There are", calls.shape[0], "calls in the table.")

There are 4490 unique case numbers.
There are 4490 calls in the table.


In [37]:
calls['CASENO'].sort_values().reset_index(drop=True).iplot(
    yTitle="Case Number", xTitle="Location in File")

In [38]:
calls['CASENO'].sort_values().head()

2747    22001097
3432    22007237
1103    22009526
3801    22017398
4079    22017403
Name: CASENO, dtype: int64

In [39]:
calls.iloc[[4121]]

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day,Lat,Lon
4121,22090846,BURGLARY AUTO,07/17/2022 12:00:00 AM,2:00:00 PM,BURGLARY - VEHICLE,0,10/13/2022 04:42:00 PM,"2900 BLOCK COLLEGE AVE\nBerkeley, CA\n(37.8582...",2900 BLOCK COLLEGE AVE,Berkeley,CA,Sunday,37.85824,-122.25317


In [40]:
calls.head(3)

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day,Lat,Lon
0,22036516,THEFT FROM AUTO,08/10/2022 12:00:00 AM,6:10:00 AM,LARCENY - FROM VEHICLE,3,10/13/2022 04:42:00 PM,"Berkeley, CA\n(37.86988, -122.27054)",NaN,Berkeley,CA,Wednesday,37.86988,-122.27054
1,22027674,BURGLARY RESIDENTIAL,06/16/2022 12:00:00 AM,8:25:00 PM,BURGLARY - RESIDENTIAL,4,10/13/2022 04:42:00 PM,"1700 BLOCK OXFORD ST\nBerkeley, CA\n(37.87674,...",1700 BLOCK OXFORD ST,Berkeley,CA,Thursday,37.87674,-122.26665
2,22023993,VANDALISM,05/25/2022 12:00:00 AM,8:00:00 AM,VANDALISM,3,10/13/2022 04:42:00 PM,"2400 BLOCK 8TH ST\nBerkeley, CA\n(37.86206, -1...",2400 BLOCK 8TH ST,Berkeley,CA,Wednesday,37.86206,-122.29346


In [41]:
calls["EVENTDT"][0]

'08/10/2022 12:00:00 AM'

In [42]:
dates = pd.to_datetime(calls["EVENTDT"])
dates[0]

Timestamp('2022-08-10 00:00:00')

In [43]:
pd.DataFrame(dict(transformed=dates, original=calls["EVENTDT"])).head()

,transformed,original
0,2022-08-10,08/10/2022 12:00:00 AM
1,2022-06-16,06/16/2022 12:00:00 AM
2,2022-05-25,05/25/2022 12:00:00 AM
3,2022-06-10,06/10/2022 12:00:00 AM
4,2022-04-20,04/20/2022 12:00:00 AM


In [44]:
times = pd.to_datetime(calls["EVENTTM"]).dt.time
times.head()

0    06:10:00
1    20:25:00
2    08:00:00
3    21:35:00
4    15:31:00
Name: EVENTTM, dtype: object

In [45]:
from datetime import datetime
timestamps = pd.concat([dates, times], axis=1).apply(
    lambda r: datetime.combine(r['EVENTDT'], r['EVENTTM']), axis=1)
timestamps.head()

0   2022-08-10 06:10:00
1   2022-06-16 20:25:00
2   2022-05-25 08:00:00
3   2022-06-10 21:35:00
4   2022-04-20 15:31:00
dtype: datetime64[ns]

In [46]:
calls['timestamp'] = timestamps
calls.head()

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day,Lat,Lon,timestamp
0,22036516,THEFT FROM AUTO,08/10/2022 12:00:00 AM,6:10:00 AM,LARCENY - FROM VEHICLE,3,10/13/2022 04:42:00 PM,"Berkeley, CA\n(37.86988, -122.27054)",NaN,Berkeley,CA,Wednesday,37.86988,-122.27054,2022-08-10 06:10:00
1,22027674,BURGLARY RESIDENTIAL,06/16/2022 12:00:00 AM,8:25:00 PM,BURGLARY - RESIDENTIAL,4,10/13/2022 04:42:00 PM,"1700 BLOCK OXFORD ST\nBerkeley, CA\n(37.87674,...",1700 BLOCK OXFORD ST,Berkeley,CA,Thursday,37.87674,-122.26665,2022-06-16 20:25:00
2,22023993,VANDALISM,05/25/2022 12:00:00 AM,8:00:00 AM,VANDALISM,3,10/13/2022 04:42:00 PM,"2400 BLOCK 8TH ST\nBerkeley, CA\n(37.86206, -1...",2400 BLOCK 8TH ST,Berkeley,CA,Wednesday,37.86206,-122.29346,2022-05-25 08:00:00
3,22026547,DISTURBANCE,06/10/2022 12:00:00 AM,9:35:00 PM,DISORDERLY CONDUCT,5,10/13/2022 04:42:00 PM,"1300 BLOCK HEARST AVE\nBerkeley, CA\n(37.87133...",1300 BLOCK HEARST AVE,Berkeley,CA,Friday,37.87133,-122.28656,2022-06-10 21:35:00
4,22018126,ASSAULT/BATTERY FEL.,04/20/2022 12:00:00 AM,3:31:00 PM,ASSAULT,3,10/13/2022 04:42:00 PM,"3000 BLOCK SHATTUCK AVE\nBerkeley, CA\n(37.855...",3000 BLOCK SHATTUCK AVE,Berkeley,CA,Wednesday,37.85507,-122.26677,2022-04-20 15:31:00


In [47]:
calls['timestamp'].min()

Timestamp('2022-04-16 00:00:00')

In [48]:
calls['timestamp'].max()

Timestamp('2022-10-12 20:30:00')

In [49]:
dow = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
calls.groupby('Day')['CASENO'].count()[dow].iplot(kind='bar', yTitle="Count")

In [50]:
calls['hour_of_day'] = (
    calls['timestamp'].dt.hour * 60 + calls['timestamp'].dt.minute ) / 60.

In [52]:
py.iplot(ff.create_distplot([calls['hour_of_day']],group_labels=["Hour"],bin_size=1, show_rug=False))

In [53]:
px.violin(calls.sort_values("CVDOW"), y="hour_of_day", x="Day", box=True, points="all", hover_name="CVLEGEND")

In [54]:
calls.head()

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day,Lat,Lon,timestamp,hour_of_day
0,22036516,THEFT FROM AUTO,08/10/2022 12:00:00 AM,6:10:00 AM,LARCENY - FROM VEHICLE,3,10/13/2022 04:42:00 PM,"Berkeley, CA\n(37.86988, -122.27054)",NaN,Berkeley,CA,Wednesday,37.86988,-122.27054,2022-08-10 06:10:00,6.166667
1,22027674,BURGLARY RESIDENTIAL,06/16/2022 12:00:00 AM,8:25:00 PM,BURGLARY - RESIDENTIAL,4,10/13/2022 04:42:00 PM,"1700 BLOCK OXFORD ST\nBerkeley, CA\n(37.87674,...",1700 BLOCK OXFORD ST,Berkeley,CA,Thursday,37.87674,-122.26665,2022-06-16 20:25:00,20.416667
2,22023993,VANDALISM,05/25/2022 12:00:00 AM,8:00:00 AM,VANDALISM,3,10/13/2022 04:42:00 PM,"2400 BLOCK 8TH ST\nBerkeley, CA\n(37.86206, -1...",2400 BLOCK 8TH ST,Berkeley,CA,Wednesday,37.86206,-122.29346,2022-05-25 08:00:00,8.000000
3,22026547,DISTURBANCE,06/10/2022 12:00:00 AM,9:35:00 PM,DISORDERLY CONDUCT,5,10/13/2022 04:42:00 PM,"1300 BLOCK HEARST AVE\nBerkeley, CA\n(37.87133...",1300 BLOCK HEARST AVE,Berkeley,CA,Friday,37.87133,-122.28656,2022-06-10 21:35:00,21.583333
4,22018126,ASSAULT/BATTERY FEL.,04/20/2022 12:00:00 AM,3:31:00 PM,ASSAULT,3,10/13/2022 04:42:00 PM,"3000 BLOCK SHATTUCK AVE\nBerkeley, CA\n(37.855...",3000 BLOCK SHATTUCK AVE,Berkeley,CA,Wednesday,37.85507,-122.26677,2022-04-20 15:31:00,15.516667


In [55]:
calls['OFFENSE'].value_counts().iplot(kind="bar")

In [56]:
calls['CVLEGEND'].value_counts().iplot(kind="bar")

In [57]:
boxes = [(len(df), go.Violin(y=df["hour_of_day"], name=i)) for (i, df) in calls.groupby("CVLEGEND")]
py.iplot([r[1] for r in sorted(boxes, key=lambda x:x[0], reverse=True)])

In [58]:
py.iplot(ff.create_distplot([
    calls[calls['CVLEGEND'] == "NOISE VIOLATION"]['hour_of_day'],
    calls[calls['CVLEGEND'] == "DRUG VIOLATION"]['hour_of_day'],
    calls[calls['CVLEGEND'] == "LIQUOR LAW VIOLATION"]['hour_of_day'],
    calls[calls['CVLEGEND'] == "FRAUD"]['hour_of_day']
],
    group_labels=["Noise Violation", "Drug Violation", "Liquor Violation", "Fraud"], 
    ))

In [59]:
calls["missing_lat_lon"] = calls[['Lat', 'Lon']].isnull().any(axis=1)
calls.groupby("CVLEGEND")['missing_lat_lon'].mean().sort_values().iplot(kind="barh")

In [63]:
import folium
import folium.plugins # The Folium Javascript Map Library

SF_COORDINATES = (37.87, -122.28)
sf_map = folium.Map(location=SF_COORDINATES, zoom_start=13)
locs = calls[['Lat', 'Lon']].astype('float').dropna().to_numpy()
heatmap = folium.plugins.HeatMap(locs.tolist(), radius = 10)
sf_map.add_child(heatmap)

In [64]:
cluster = folium.plugins.MarkerCluster()
for _, r in calls[['Lat', 'Lon', 'CVLEGEND']].tail(1000).dropna().iterrows():
    cluster.add_child(
        folium.Marker([float(r["Lat"]), float(r["Lon"])], popup=r['CVLEGEND']))
    
sf_map = folium.Map(location=SF_COORDINATES, zoom_start=13)
sf_map.add_child(cluster)
sf_map